In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Loader

In [2]:
import torch
import numpy as np
import pandas as pd
def prepare_data_main_model(df, seq_length, output_size, date_range):
    '''
    df: pandas df contain all the data
    seq_length: number of days consider as input
    output_size: number of days to predict
    date_range: length of history to consider
    Output:
    - full_data: Prepared data as a list of tuples (input_sequence, target_labels).
    - state_ordered: List of unique states in the data.
    '''
    full_data = []
    state_ordered = []

    for state in df.index.get_level_values('fips').unique():
        df_state = df.iloc[df.index.get_level_values('fips') == state]
 # If the length of data is less than or equal to date_range, consider all data
        if len(df_state) <= date_range:
            L = len(df_state.to_numpy())
            train_state = []
            for i in range(L - seq_length - output_size + 1):
                train_seq = df_state.to_numpy()[i:i + seq_length]
                train_label = df_state.to_numpy()[i:i + seq_length + output_size][seq_length:seq_length + output_size,
                              0]
                train_state.append((train_seq, train_label))

            for x in train_state:
                full_data.append(x)
            state_ordered.append(state)
# If data is larger than date_range, consider the most recent date_range data
        else:
            df_state = df.iloc[df.index.get_level_values('fips') == state][-date_range:]

            train_state = []

            L = len(df_state.to_numpy())
            for i in range(L - seq_length - output_size + 1):
                train_seq = df_state.to_numpy()[i:i + seq_length]
                train_label = df_state.to_numpy()[i:i + seq_length + output_size][seq_length:seq_length + output_size,
                              0]
                train_state.append((train_seq, train_label))

            for x in train_state:
                full_data.append(x)
            state_ordered.append(state)
    return full_data, state_ordered

def splitdata(full_data, ratio, batch_size):
    '''
    Split the prepared data into training and testing datasets.

    Parameters:
    - full_data: Prepared data as a list of tuples (input_sequence, target_labels).
    - ratio: Ratio of data to be used for training (e.g., 0.8 for 80% training, 20% testing).
    - batch_size: Size of mini-batches for DataLoader.

    Output:
    - train_loader: DataLoader for the training dataset.
    - test_loader: DataLoader for the testing dataset.
    '''
    train_size = int(ratio * len(full_data))
    test_size = len(full_data) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(full_data, [train_size, test_size])

    train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                               batch_size=batch_size,
                                               shuffle=True, drop_last=True)

    test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                              batch_size=batch_size,
                                              shuffle=True, drop_last=True)

    return train_loader, test_loader

#gd = pd.read_csv('/content/drive/MyDrive/Flu Forecasting/code/groundtruth.csv')
gd = pd.read_csv('/content/drive/MyDrive/Flu Forecasting/code/groundtruth2m.csv')
gd['Week_end']  = pd.to_datetime(gd['Week_end'])


# Models

In [3]:
import torch
import pandas as pd
from torch import nn
import torch.nn.functional as F
pd.options.mode.chained_assignment = None

## LSTM

### Target: Rate ( rate = total admission * 100000/population) Current Version!!!

In [ ]:


class LSTM(nn.Module):

    def __init__(self, input_size, hidden_layer_size, num_layers, output_size, dropout_rate):
        super().__init__()

        self.input_size = input_size

        self.hidden_layer_size = hidden_layer_size

        self.num_layers = num_layers

        self.output_size = output_size

        self.lstm = nn.LSTM(self.input_size, hidden_layer_size, num_layers, batch_first=True, dropout=dropout_rate)

        self.linear = nn.Linear(hidden_layer_size, 1000)

        self.dropout = nn.Dropout(dropout_rate)

        self.linear2 = nn.Linear(1000, output_size)

    def forward(self, input_seq):
        h = (torch.zeros(self.num_layers, input_seq.size(0), self.hidden_layer_size).to(device),
             torch.zeros(self.num_layers, input_seq.size(0), self.hidden_layer_size).to(device))

        lstm_out, self.hidden_cell = self.lstm(input_seq, h)

        # only return the results for last sequence
        lstm_out = lstm_out[:, -1, :]
        predictions = self.linear(lstm_out)
        #predictions = F.relu(predictions)
        #predictions = F.softplus(predictions)
        predictions = F.leaky_relu(predictions, negative_slope=0.01)
        predictions = self.dropout(predictions)
        predictions = self.linear2(predictions)

        return predictions

#### Main


In [ ]:
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import MinMaxScaler
from torch import nn
from tqdm import trange
from datetime import date, timedelta

from collections import OrderedDict
from collections import namedtuple
from itertools import product

import torch.nn.functional as F


data_file = '/content/drive/MyDrive/Flu Forecasting/code/trainingdata_rate0131.csv'
model_file = '/content/drive/MyDrive/Flu Forecasting/LSTM/weightselection/'
prediction_file = '/content/drive/MyDrive/Flu Forecasting/LSTM/weightselection/'
summary_file ='/content/drive/MyDrive/Flu Forecasting/LSTM/weightselection/'
def perdelta(start, end, delta):
    curr = start
    while curr < end:
        yield curr
        curr += delta
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



params = OrderedDict(
    target = ['rate'], ### Our target variable
    lr = [0.0001,0.00025],
    batch_size = [2],
    seq_length = [6,8,12], # length of input sequence to predict values
    output_size = [4], # we are predicting next four weeks admissions
    num_pred_features = [1],
    week_range = [32,64,128], # number of historical data (per state) used to train the model
    input_size = [15], # total number of input features
    hidden_layer_size = [64,128,256], #test
    num_layers = [1], #test
    ratio = [0.7],  # ratio of training set and validation set
    num_epochs = [50],
    dropout_rate = [0.8],
    lossfunc = [nn.SmoothL1Loss(beta=0.05, reduction = 'sum'),

                ]
)


class RunBuilder():
    @staticmethod
    def get_runs(params):

        Run = namedtuple('Run', params.keys())

        runs = []
        for v in product(*params.values()):
            runs.append(Run(*v))

        return runs

pd.options.mode.chained_assignment = None

runs = RunBuilder.get_runs(params)

# output dict
columns = [
    'Model',
    'lr',
    'batch_size',
    'seq_length',
    'week_range',
    'hidden_layer_size',
    'num_layers',
    'lossfunc',
    'mse_validation',
    'mse_validation_1w',
    'mse_validation_2w',
    'mse_validation_3w',
    'mse_validation_4w',
    'Target'
]
# output dataframe
df_summary = pd.DataFrame(columns=columns)
for run in RunBuilder.get_runs(params):
    if run.target == 'rate':
        results_file = 'rate'

    validation_predictions = []
    validation_labels = []
    # read data
    df = pd.read_csv(data_file)
    #df = df.rename(columns={'End Date': 'Week_end', 'State': 'fips'})
    df['Week_end'] = pd.to_datetime(df['Week_end'])
    # too many missing values in State'11'
    df =  df[df['fips'] != 11]
    #df =  df[df['fips'] == 1]
    #df = df[['fips', 'Week_end', 'total_admissions']]

    # drop weather data
    #df = df.drop(['PRCP_mean', 'SNOW_mean', 'TMAXDELTA', 'TMINDELTA'], axis=1)
    '''
    # choose data in 2023-11 as test data  test 10-01
    #start_date1 = pd.to_datetime('2023-11-01') - timedelta(weeks=run.seq_length * 7)
    df_test = df[(df['Week_end'] > start_date1) & (df['Week_end'] < pd.to_datetime('2023-11-26'))] # Use data after 2023-11-01 as testing set
    df_test = df_test.set_index(['fips', 'Week_end'])
    df_test1 = df_test.copy()
    #choose data before 2023-11 as training data and validation data
    df_train = df[(df['Week_end'] <= pd.to_datetime('2023-11-01'))& (df['Week_end'] >pd.to_datetime('2022-08-01')) ] # Use data before 11/1 as training set
    df_train = df_train.set_index(['fips', 'Week_end'])
    '''
    # test set is the week from 2024-01-13
    start_date1 = pd.to_datetime('2024-01-20') - timedelta(weeks=run.seq_length)
    df_test = df[(df['Week_end'] > start_date1) & (df['Week_end'] <pd.to_datetime('2024-01-30'))] # Use data after 2023-11-01 as testing set
    df_test = df_test.set_index(['fips', 'Week_end'])
    df_test1 = df_test.copy()
    #choose data before 2024-01-06 as training data and validation data and after 2022-08-01
    df_train = df[(df['Week_end'] <= pd.to_datetime('2024-01-20'))& (df['Week_end'] >pd.to_datetime('2022-08-01')) ] # Use data before 11/1 as training set

    df_train = df_train.set_index(['fips', 'Week_end'])

    #rescale the data from 0 to 1, normalization
    first_col = df_train.pop(run.target)
    df_train.insert(0, run.target, first_col)
    first_col_2 = df_test.pop(run.target)
    df_test.insert(0, run.target, first_col_2)

    scaler = MinMaxScaler(feature_range=(0, 1))

    scaler.fit(df_train.iloc[:, 1:])
    train_features_normalized = scaler.transform(df_train.iloc[:, 1:])
    test_feature_normalized = scaler.transform(df_test.iloc[:, 1:])

    scaler_target = MinMaxScaler(feature_range=(0, 1))
    scaler_target.fit(np.asarray(df_train.iloc[:, 0]).reshape(-1, 1))
    train_target_normalized = scaler_target.transform(np.asarray(df_train.iloc[:, 0]).reshape(-1, 1))
    test_target_normalized = scaler_target.transform(np.asarray(df_test.iloc[:, 0]).reshape(-1, 1))

    df_train.iloc[:, 1:] = train_features_normalized
    df_train.iloc[:, 0] = train_target_normalized
    df_test.iloc[:, 1:] = test_feature_normalized
    df_test.iloc[:, 0] = test_target_normalized

    '''
    Training
    '''
    # prepare the training and validation data (sequance format)
    full_data_main, state_ordered = prepare_data_main_model(df_train, run.seq_length,
                                                            run.output_size, run.week_range)

    model_main = LSTM(run.input_size, run.hidden_layer_size, run.num_layers, run.output_size ,
                      run.dropout_rate).to(device)
    # split the training data into training set and validation set
    train_loader_main, test_loader_main = splitdata(full_data_main, run.ratio, run.batch_size)

    loss_function = run.lossfunc
  # add l2 regularization
    optimizer_main = torch.optim.Adam(model_main.parameters(), lr=run.lr,weight_decay=1e-5)

    track_loss_train = []
    track_loss_test = []
    best_loss = 100000
    ###

    best_mse_numerical = None
    mse_numerical_weekly = [None] * 4

    best_mse_numerical_weekly = [None] * 4
    for i in trange(run.num_epochs):
# Set the model in training mode
        model_main.train()
        epoch_loss_train = 0
# Iterate over the training data
        for i, (seq, labels) in enumerate(train_loader_main):
            seq, labels = seq.to(device), labels.to(device)
            optimizer_main.zero_grad()
            seq = torch.as_tensor(seq).reshape(-1, run.seq_length, run.input_size)
             # Initialize the hidden state of the LSTM
            model_main.hidden_cell = (torch.zeros(run.num_layers, seq.size()[0], run.hidden_layer_size).to(device),
                          torch.zeros(run.num_layers, seq.size()[0], run.hidden_layer_size).to(device))

            # Forward pass to get predictions
            y_pred = model_main(seq.float())

            # Calculate the loss and perform backpropagation
            single_loss = loss_function(y_pred, torch.as_tensor(labels).float())
            single_loss.backward()
            optimizer_main.step()

            epoch_loss_train += single_loss.item()
        # Track the training loss for this epoch
        track_loss_train.append(epoch_loss_train)

        with torch.no_grad():
          # Set the model in evaluation mode
            model_main.eval()
            epoch_loss_test = 0
            ########
            # Initialize lists to store validation predictions and labels
            validation_predictions = []
            validation_labels = []
            for i, (seq, labels) in enumerate(test_loader_main):
                seq, labels = seq.to(device), labels.to(device)
                seq = torch.as_tensor(seq).reshape(-1, run.seq_length, run.input_size)
                model_main.hidden_cell = (torch.zeros(run.num_layers, seq.size(0), run.hidden_layer_size).to(device),
                                 torch.zeros(run.num_layers, seq.size(0), run.hidden_layer_size).to(device))
                y_pred = model_main(seq.float())

                single_loss = loss_function(y_pred, torch.as_tensor(labels).float())
                epoch_loss_test += single_loss.item()
                ######
                # Store predictions and labels for later evaluation
                validation_predictions.extend(y_pred.cpu().numpy())
                validation_labels.extend(labels.cpu().numpy())
            track_loss_test.append(epoch_loss_test)
        # Combine validation predictions and labels, and perform inverse scaling to get true value
        validation_predictions = np.concatenate(validation_predictions, axis=0)
        validation_labels = np.concatenate(validation_labels, axis=0)
        validation_predictions = scaler_target.inverse_transform(validation_predictions.reshape(-1, 1)).reshape(-1, 4)
        validation_labels = scaler_target.inverse_transform(validation_labels.reshape(-1, 1)).reshape(-1, 4)

        # Calculate total and weekly MSE and accuracy for validation set
        mse_numerical = np.mean((validation_predictions - validation_labels)**2)

        for week in range(4):
            # Calculate weekly MSE
            mse_numerical_week = np.mean((validation_predictions[:, week] - validation_labels[:, week])**2)
            mse_numerical_weekly[week] = mse_numerical_week


        if epoch_loss_test  < best_loss:
          # Update the best loss and save the model; perform early stopping
            best_mse_numerical = mse_numerical
            best_mse_numerical_weekly = mse_numerical_weekly

            best_loss = epoch_loss_test
            print('Train Loss: ', epoch_loss_train)
            print('Test Loss: ', epoch_loss_test)
            es = 0
            torch.save(model_main.state_dict(),
                           model_file + run.target +'_' +str(run.lr) + '_' +
                           str(run.week_range) + '_' + str(run.hidden_layer_size)\
                           + '_' + str(run.seq_length) + '_' + str(run.num_layers)\
                           + '_LSTM_weights.pt')
        else:
            es += 1
            print("Counter {} of 5".format(es))
            print('Train Loss: ', epoch_loss_train)
            print('Test Loss: ', epoch_loss_test)


        if es > 4:
            print("Early stopping with best_loss: ", best_loss, "and test_loss for this epoch: ",
                      epoch_loss_test,
                      "...")

            break

    '''
    Prediction
    '''
    df_output = pd.DataFrame(columns=['fips', 'Week_end', 'Prediction_1w',
                                      'Prediction_2w', 'Prediction_3w',
                                      'Prediction_4w'])
  #test set starts with 2023-11-01
    test_weeks = df_test[df_test.index.get_level_values('Week_end') \
                       >= pd.to_datetime('2024-01-20')].index.get_level_values('Week_end').unique()
    test_states = df_test.index.get_level_values('fips').unique()

    m_state_dict_main = torch.load(model_file + run.target+ '_' +str(run.lr) + '_' +
                                   str(run.week_range) + '_' + str(run.hidden_layer_size) \
                                   + '_' + str(run.seq_length) + '_' + str(run.num_layers) \
                                   + '_LSTM_weights.pt')

    model_main = LSTM(run.input_size, run.hidden_layer_size, run.num_layers,
                      run.output_size, run.dropout_rate).to(device)
    model_main.load_state_dict(m_state_dict_main)

    with torch.no_grad():
        #model_main.eval()
        for fips in test_states:
            for week in test_weeks:
                seq = df_test[(df_test.index.get_level_values('fips') == fips)\
                 & (df_test.index.get_level_values('Week_end') <= week)][-run.seq_length:].to_numpy()

                seq = torch.tensor(seq).reshape(-1, run.seq_length, run.input_size).to(device)


                model_main.hidden_cell = (torch.zeros(run.num_layers, seq.size(0), run.hidden_layer_size).to(device),
                                 torch.zeros(run.num_layers, seq.size(0), run.hidden_layer_size).to(device))
                prediction = model_main(seq.float())

                prediction = scaler_target.inverse_transform(prediction.cpu().detach().numpy().reshape(-1, 1))
                # save prediction results
                dic = {
                    'fips' : fips,
                    'Week_end' : week,
                    'Prediction_1w' : prediction[0].item(),
                    'Prediction_2w' : prediction[1].item(),
                    'Prediction_3w' : prediction[2].item(),
                    'Prediction_4w' : prediction[3].item()

                }

                df_output = pd.concat([df_output, pd.DataFrame([dic])], ignore_index=True)


        df_output.to_csv(prediction_file + run.target + '_' +str(run.lr) + '_' +
                            str(run.week_range) + '_' + str(run.hidden_layer_size) \
                            + '_' + str(run.seq_length) + '_' + str(run.num_layers) \
                            + '.csv')

        '''
        Evaluation
        '''





        dic_lstm = {
            'Model': 'LSTM',
            'lr': run.lr,
            'batch_size': run.batch_size,
            'seq_length': run.seq_length,
            'week_range': run.week_range,
            'hidden_layer_size': run.hidden_layer_size,
            'num_layers': run.num_layers,
            'lossfunc': run.lossfunc,

            'mse_validation': best_mse_numerical,

            'mse_validation_1w': best_mse_numerical_weekly[0],
            'mse_validation_2w': best_mse_numerical_weekly[1],
            'mse_validation_3w': best_mse_numerical_weekly[2],
            'mse_validation_4w': best_mse_numerical_weekly[3],

            'Target': run.target
        }

        print('mse_validation:')
        print(dic_lstm['mse_validation'])


        df_summary = pd.concat([df_summary, pd.DataFrame([dic_lstm])], ignore_index=True)

        df_summary.to_csv(summary_file + 'summarytest.csv')



/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.8 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
  2%|▏         | 1/50 [00:01<00:58,  1.19s/it]

Train Loss:  186.36368022114038
Test Loss:  50.24806038476527


  4%|▍         | 2/50 [00:02<00:57,  1.19s/it]

Train Loss:  152.66013035550714
Test Loss:  43.47965035121888


  6%|▌         | 3/50 [00:03<00:54,  1.17s/it]

...

Counter 4 of 5
Train Loss:  123.33088021795265
Test Loss:  45.79262294335058


 36%|███▌      | 18/50 [01:04<01:54,  3.58s/it]

Counter 5 of 5
Train Loss:  127.9452423696639
Test Loss:  40.75564348141779
Early stopping with best_loss:  40.051624792598886 and test_loss for this epoch:  40.75564348141779 ...



/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.8 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


mse_validation:
0.9096556672484392


In [ ]:
df_output

,fips,Week_end,Prediction_1w,Prediction_2w,Prediction_3w,Prediction_4w
0,1,2024-01-20,1.553443,1.416178,1.360365,1.205082
1,1,2024-01-27,1.487583,1.222604,1.125432,1.034163
2,2,2024-01-20,0.780145,1.242551,1.931762,2.828534
3,2,2024-01-27,0.874534,1.304484,1.812373,1.968853
4,4,2024-01-20,4.193772,2.836079,2.034209,1.963451
...,...,...,...,...,...,...
95,54,2024-01-27,3.249407,3.569684,3.140540,2.866554
96,55,2024-01-20,2.217776,2.846013,3.614674,3.190379
97,55,2024-01-27,2.407753,2.235549,2.041720,1.472630
98,56,2024-01-20,4.787724,3.387617,3.147089,3.133203


#### Run best model for 100 times

In [ ]:
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import MinMaxScaler
from torch import nn
from tqdm import trange
from datetime import date, timedelta

from collections import OrderedDict
from collections import namedtuple
from itertools import product

import torch.nn.functional as F


n_result = pd.DataFrame()
data_file = '/content/drive/MyDrive/Flu Forecasting/code/trainingdata_rate0131.csv'
model_file = '/content/drive/MyDrive/Flu Forecasting/LSTM/weightselection/'
prediction_file = '/content/drive/MyDrive/Flu Forecasting/LSTM/weightselection/'
summary_file ='/content/drive/MyDrive/Flu Forecasting/LSTM/weightselection/'
# read data
def perdelta(start, end, delta):
    curr = start
    while curr < end:
        yield curr
        curr += delta
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



params = OrderedDict(
    target = ['rate'], ### Our target variable
    lr = [0.0001],
    batch_size = [2],
    seq_length = [8], # length of input sequence to predict values
    output_size = [4], # we are predicting next four weeks admissions
    num_pred_features = [1],
    week_range = [64], # number of historical data (per state) used to train the model
    input_size = [15], # total number of input features
    hidden_layer_size = [64], #test
    num_layers = [1], #test
    ratio = [0.7],  # ratio of training set and validation set
    num_epochs = [50],
    dropout_rate = [0.8],
    lossfunc = [nn.SmoothL1Loss(beta=0.05, reduction = 'sum'),

                ]
)


class RunBuilder():
    @staticmethod
    def get_runs(params):

        Run = namedtuple('Run', params.keys())

        runs = []
        for v in product(*params.values()):
            runs.append(Run(*v))

        return runs

pd.options.mode.chained_assignment = None

runs = RunBuilder.get_runs(params)

for run in RunBuilder.get_runs(params):
    if run.target == 'rate':
        results_file = 'rate'

    validation_predictions = []
    validation_labels = []
    # read data
    df = pd.read_csv(data_file)
    #df = df.rename(columns={'End Date': 'Week_end', 'State': 'fips'})
    df['Week_end'] = pd.to_datetime(df['Week_end'])
    # too many missing values in State'11'
    df =  df[df['fips'] != 11]
    #df =  df[df['fips'] == 1]
    #df = df[['fips', 'Week_end', 'total_admissions']]
    #df = df.drop(['PRCP_mean', 'SNOW_mean', 'TMAXDELTA', 'TMINDELTA'], axis=1)
    # choose data in 2023-11 as test data  test 10-01

    '''
    start_date1 = pd.to_datetime('2023-11-01') - timedelta(weeks=run.seq_length * 7)
    df_test = df[(df['Week_end'] > start_date1) & (df['Week_end'] < pd.to_datetime('2023-11-26'))] # Use data after 2023-11-01 as testing set
    df_test = df_test.set_index(['fips', 'Week_end'])
    df_test1 = df_test.copy()
    #choose data before 2023-11 as training data and validation data
    df_train = df[df['Week_end'] <= pd.to_datetime('2023-11-01')] # Use data before 11/1 as training set
    df_train = df_train.set_index(['fips', 'Week_end'])

    #rescale the data from 0 to 1, normalization
    first_col = df_train.pop(run.target)
    df_train.insert(0, run.target, first_col)
    first_col_2 = df_test.pop(run.target)
    df_test.insert(0, run.target, first_col_2)
    '''
    start_date1 = pd.to_datetime('2024-01-20') - timedelta(weeks=run.seq_length)
    df_test = df[(df['Week_end'] > start_date1) & (df['Week_end'] <pd.to_datetime('2024-01-26'))] # Use data after 2023-11-01 as testing set
    df_test = df_test.set_index(['fips', 'Week_end'])
    df_test1 = df_test.copy()
    #choose data before 2024-01-13 as training data and validation data
    df_train = df[(df['Week_end'] <= pd.to_datetime('2024-01-20'))& (df['Week_end'] >pd.to_datetime('2022-08-01')) ] # Use data before 11/1 as training set
    df_train = df_train.set_index(['fips', 'Week_end'])

    #rescale the data from 0 to 1, normalization
    first_col = df_train.pop(run.target)
    df_train.insert(0, run.target, first_col)
    first_col_2 = df_test.pop(run.target)
    df_test.insert(0, run.target, first_col_2)

    scaler = MinMaxScaler(feature_range=(0, 1))

    scaler.fit(df_train.iloc[:, 1:])
    train_features_normalized = scaler.transform(df_train.iloc[:, 1:])
    test_feature_normalized = scaler.transform(df_test.iloc[:, 1:])

    scaler_target = MinMaxScaler(feature_range=(0, 1))
    scaler_target.fit(np.asarray(df_train.iloc[:, 0]).reshape(-1, 1))
    train_target_normalized = scaler_target.transform(np.asarray(df_train.iloc[:, 0]).reshape(-1, 1))
    test_target_normalized = scaler_target.transform(np.asarray(df_test.iloc[:, 0]).reshape(-1, 1))

    df_train.iloc[:, 1:] = train_features_normalized
    df_train.iloc[:, 0] = train_target_normalized
    df_test.iloc[:, 1:] = test_feature_normalized
    df_test.iloc[:, 0] = test_target_normalized


    for i in range(1000):
      df_output = pd.DataFrame(columns=['fips', 'Week_end', 'Prediction_1w',
                                        'Prediction_2w', 'Prediction_3w',
                                        'Prediction_4w'])
    #test set starts with 2024-01-13
      test_weeks = df_test[df_test.index.get_level_values('Week_end') \
                          >= pd.to_datetime('2024-01-20')].index.get_level_values('Week_end').unique()
      test_states = df_test.index.get_level_values('fips').unique()

      m_state_dict_main = torch.load(model_file + run.target + '_' +str(run.lr) + '_' +
                                      str(run.week_range) + '_' + str(run.hidden_layer_size) \
                                      + '_' + str(run.seq_length) + '_' + str(run.num_layers) \
                                      + '_LSTM_weights.pt')

      model_main = LSTM(run.input_size, run.hidden_layer_size, run.num_layers,
                        run.output_size, run.dropout_rate).to(device)
      model_main.load_state_dict(m_state_dict_main)

      with torch.no_grad():
          #model_main.eval()
          for fips in test_states:
              for week in test_weeks:
                  seq = df_test[(df_test.index.get_level_values('fips') == fips)\
                    & (df_test.index.get_level_values('Week_end') <= week)][-run.seq_length:].to_numpy()

                  seq = torch.tensor(seq).reshape(-1, run.seq_length, run.input_size).to(device)


                  model_main.hidden_cell = (torch.zeros(run.num_layers, seq.size(0), run.hidden_layer_size).to(device),
                                    torch.zeros(run.num_layers, seq.size(0), run.hidden_layer_size).to(device))
                  prediction = model_main(seq.float())

                  prediction = scaler_target.inverse_transform(prediction.cpu().detach().numpy().reshape(-1, 1))
                  # save prediction results
                  dic = {
                      'fips' : fips,
                      'Week_end' : week,
                      'Prediction_1w' : prediction[0].item(),
                      'Prediction_2w' : prediction[1].item(),
                      'Prediction_3w' : prediction[2].item(),
                      'Prediction_4w' : prediction[3].item()

                  }

                  df_output = pd.concat([df_output, pd.DataFrame([dic])], ignore_index=True)


          df_output.to_csv(prediction_file + run.target + '_' +str(run.lr) + '_' +
                              str(run.week_range) + '_' + str(run.hidden_layer_size) \
                              + '_' + str(run.seq_length) + '_' + str(run.num_layers) \
                              + '.csv')
          n_result= pd.concat([n_result, df_output], ignore_index=True)


    n_result.to_pickle(summary_file + 'allresulttest4.pkl')

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.8 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.8 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.8 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout op

In [ ]:
n_result

,fips,Week_end,Prediction_1w,Prediction_2w,Prediction_3w,Prediction_4w
0,1,2023-12-16,2.030811,3.404916,4.509559,5.837484
1,2,2023-12-16,0.023632,0.239356,0.150441,0.217191
2,4,2023-12-16,5.346867,8.291306,8.709900,7.924041
3,5,2023-12-16,1.159860,2.735837,3.255077,4.091725
4,6,2023-12-16,1.004027,1.336492,1.735048,1.567238
...,...,...,...,...,...,...
49995,51,2023-12-16,2.396512,3.624523,3.849347,3.313382
49996,53,2023-12-16,0.184649,0.052782,0.142393,0.099187
49997,54,2023-12-16,0.541724,1.201434,1.669058,2.020296
49998,55,2023-12-16,1.475421,3.360832,4.460989,4.914268


In [ ]:
from google.colab import runtime
runtime.unassign()

## GRU

In [5]:
class GRU(nn.Module):

    def __init__(self, input_size, hidden_layer_size, num_layers, output_size, dropout_rate):
        super().__init__()

        self.input_size = input_size
        self.hidden_layer_size = hidden_layer_size
        self.num_layers = num_layers
        self.output_size = output_size


        self.gru = nn.GRU(self.input_size, hidden_layer_size, num_layers, batch_first=True, dropout=dropout_rate)

        self.linear = nn.Linear(hidden_layer_size, 1000)
        self.dropout = nn.Dropout(dropout_rate)
        self.linear2 = nn.Linear(1000, output_size)

    def forward(self, input_seq):
        h = torch.zeros(self.num_layers, input_seq.size(0), self.hidden_layer_size).to(device)


        gru_out, self.hidden = self.gru(input_seq, h)


        gru_out = gru_out[:, -1, :]
        predictions = self.linear(gru_out)
        predictions = F.relu(predictions)
        predictions = self.dropout(predictions)
        predictions = self.linear2(predictions)

        return predictions


### Main

In [6]:
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import MinMaxScaler
from torch import nn
from tqdm import trange
from datetime import date, timedelta

from collections import OrderedDict
from collections import namedtuple
from itertools import product

import torch.nn.functional as F


'''
Don't save trained model or results in github, there is no enough space
'''
data_file = '/content/drive/MyDrive/Flu Forecasting/code/trainingdata_rate0131.csv'
model_file = '/content/drive/MyDrive/Flu Forecasting/GRU/weightselection/'
prediction_file = '/content/drive/MyDrive/Flu Forecasting/GRU/weightselection/'
summary_file ='/content/drive/MyDrive/Flu Forecasting/GRU/weightselection/'
def perdelta(start, end, delta):
    curr = start
    while curr < end:
        yield curr
        curr += delta
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


params = OrderedDict(
    target = ['rate'], ### Our target variable
    lr = [0.0001,0.00025],
    batch_size = [2],
    seq_length = [6,8,12], # length of input sequence to predict values
    output_size = [4], # we are predicting next four weeks admissions
    num_pred_features = [1],
    week_range = [32,64,128], # number of historical data (per state) used to train the model
    input_size = [15], # total number of input features
    hidden_layer_size = [64,128,256], #test
    num_layers = [1], #test
    ratio = [0.7],  # ratio of training set and validation set
    num_epochs = [50],
    dropout_rate = [0.8],
    lossfunc = [nn.SmoothL1Loss(beta=0.05, reduction = 'sum'),

                ]
)

class RunBuilder():
    @staticmethod
    def get_runs(params):

        Run = namedtuple('Run', params.keys())

        runs = []
        for v in product(*params.values()):
            runs.append(Run(*v))

        return runs

pd.options.mode.chained_assignment = None

runs = RunBuilder.get_runs(params)

# output dict
columns = [
    'Model',
    'lr',
    'batch_size',
    'seq_length',
    'week_range',
    'hidden_layer_size',
    'num_layers',
    'lossfunc',
    'mse_validation',
    'mse_validation_1w',
    'mse_validation_2w',
    'mse_validation_3w',
    'mse_validation_4w',
    'Target'
]
# output dataframe
df_summary = pd.DataFrame(columns=columns)
for run in RunBuilder.get_runs(params):
    if run.target == 'rate':
        results_file = 'rate'

    validation_predictions = []
    validation_labels = []
    # read data
    df = pd.read_csv(data_file)
    #df = df.rename(columns={'End Date': 'Week_end', 'State': 'fips'})
    df['Week_end'] = pd.to_datetime(df['Week_end'])
    # too many missing values in State'11'
    df =  df[df['fips'] != 11]
    #df =  df[df['fips'] == 1]
    #df = df[['fips', 'Week_end', 'total_admissions']]

    # drop weather data
    #df = df.drop(['PRCP_mean', 'SNOW_mean', 'TMAXDELTA', 'TMINDELTA'], axis=1)
    # choose data in 2023-11 as test data  test 10-01
    '''
    start_date1 = pd.to_datetime('2023-11-01') - timedelta(weeks=run.seq_length * 7)
    df_test = df[(df['Week_end'] > start_date1) & (df['Week_end'] < pd.to_datetime('2023-11-26'))] # Use data after 2023-11-01 as testing set
    df_test = df_test.set_index(['fips', 'Week_end'])
    df_test1 = df_test.copy()
    #choose data before 2023-11 as training data and validation data
    df_train = df[(df['Week_end'] <= pd.to_datetime('2023-11-01'))& (df['Week_end'] >pd.to_datetime('2022-08-01')) ] # Use data before 11/1 as training set
    df_train = df_train.set_index(['fips', 'Week_end'])
'''


    start_date1 = pd.to_datetime('2024-01-20') - timedelta(weeks=run.seq_length)
    df_test = df[(df['Week_end'] > start_date1) & (df['Week_end'] <pd.to_datetime('2024-01-30'))] # Use data after 2023-11-01 as testing set
    df_test = df_test.set_index(['fips', 'Week_end'])
    df_test1 = df_test.copy()
    #choose data before 2024-01-06 as training data and validation data and after 2022-08-01
    df_train = df[(df['Week_end'] <= pd.to_datetime('2024-01-20'))& (df['Week_end'] >pd.to_datetime('2022-08-01')) ] # Use data before 11/1 as training set

    df_train = df_train.set_index(['fips', 'Week_end'])
    #rescale the data from 0 to 1, normalization

    first_col = df_train.pop(run.target)
    df_train.insert(0, run.target, first_col)
    first_col_2 = df_test.pop(run.target)
    df_test.insert(0, run.target, first_col_2)

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler.fit(df_train.iloc[:, 1:])
    train_features_normalized = scaler.transform(df_train.iloc[:, 1:])
    test_feature_normalized = scaler.transform(df_test.iloc[:, 1:])

    scaler_target = MinMaxScaler(feature_range=(0, 1))
    scaler_target.fit(np.asarray(df_train.iloc[:, 0]).reshape(-1, 1))
    train_target_normalized = scaler_target.transform(np.asarray(df_train.iloc[:, 0]).reshape(-1, 1))
    test_target_normalized = scaler_target.transform(np.asarray(df_test.iloc[:, 0]).reshape(-1, 1))

    df_train.iloc[:, 1:] = train_features_normalized
    df_train.iloc[:, 0] = train_target_normalized
    df_test.iloc[:, 1:] = test_feature_normalized
    df_test.iloc[:, 0] = test_target_normalized

    '''
    Training
    '''

    full_data_main, state_ordered = prepare_data_main_model(df_train, run.seq_length,
                                                            run.output_size, run.week_range)


    model_main = GRU(run.input_size, run.hidden_layer_size, run.num_layers, run.output_size, run.dropout_rate).to(device)


    train_loader_main, test_loader_main = splitdata(full_data_main, run.ratio, run.batch_size)

    loss_function = run.lossfunc

    optimizer_main = torch.optim.Adam(model_main.parameters(), lr=run.lr)

    track_loss_train = []
    track_loss_test = []
    best_loss = 100000
    ###
    best_mse_categorical = None
    best_accuracy_categorical = None
    best_mse_numerical = None
    mse_numerical_weekly = [None] * 4
    accuracy_categorical_weekly = [None] * 4
    mse_categorical_weekly = [None] * 4
    best_mse_categorical_weekly = [None] * 4
    best_accuracy_categorical_weekly = [None] * 4
    best_mse_numerical_weekly = [None] * 4
    for i in trange(run.num_epochs):

        model_main.train()
        epoch_loss_train = 0

        for i, (seq, labels) in enumerate(train_loader_main):
            seq, labels = seq.to(device), labels.to(device)
            optimizer_main.zero_grad()
            seq = torch.as_tensor(seq).reshape(-1, run.seq_length, run.input_size)
            model_main.hidden = torch.zeros(run.num_layers, seq.size(0), run.hidden_layer_size).to(device)

            y_pred = model_main(seq.float())

            single_loss = loss_function(y_pred, torch.as_tensor(labels).float())
            single_loss.backward()
            optimizer_main.step()

            epoch_loss_train += single_loss.item()

        track_loss_train.append(epoch_loss_train)
        model_main.eval()
        with torch.no_grad():
            epoch_loss_test = 0
          ########
            validation_predictions = []
            validation_labels = []
            for i, (seq, labels) in enumerate(test_loader_main):
                seq, labels = seq.to(device), labels.to(device)
                seq = torch.as_tensor(seq).reshape(-1, run.seq_length, run.input_size)
                model_main.hidden = torch.zeros(run.num_layers, seq.size(0), run.hidden_layer_size).to(device)
                y_pred = model_main(seq.float())

                single_loss = loss_function(y_pred, torch.as_tensor(labels).float())
                epoch_loss_test += single_loss.item()

                            ######
                validation_predictions.extend(y_pred.cpu().numpy())
                validation_labels.extend(labels.cpu().numpy())
            track_loss_test.append(epoch_loss_test)

        validation_predictions = np.concatenate(validation_predictions, axis=0)
        validation_labels = np.concatenate(validation_labels, axis=0)
        validation_predictions = scaler_target.inverse_transform(validation_predictions.reshape(-1, 1)).reshape(-1, 4)
        validation_labels = scaler_target.inverse_transform(validation_labels.reshape(-1, 1)).reshape(-1, 4)

        # Calculate total and weekly MSE and accuracy
        mse_numerical = np.mean((validation_predictions - validation_labels)**2)

        for week in range(4):
            # Numerical MSE
            mse_numerical_week = np.mean((validation_predictions[:, week] - validation_labels[:, week])**2)
            mse_numerical_weekly[week] = mse_numerical_week


        if epoch_loss_test  < best_loss:
          #####
            best_mse_numerical = mse_numerical
            best_mse_numerical_weekly = mse_numerical_weekly

            best_loss = epoch_loss_test
            print('Train Loss: ', epoch_loss_train)
            print('Test Loss: ', epoch_loss_test)
            es = 0
            torch.save(model_main.state_dict(),
                           model_file + run.target + '_' + str(run.lr) + '_' +
                           str(run.week_range) + '_' + str(run.hidden_layer_size)\
                           + '_' + str(run.seq_length) + '_' + str(run.num_layers)\
                                                  + '_GRU_weights.pt')
        else:
            es += 1
            print("Counter {} of 5".format(es))
            print('Train Loss: ', epoch_loss_train)
            print('Test Loss: ', epoch_loss_test)


        if es > 4:
            print("Early stopping with best_loss: ", best_loss, "and test_loss for this epoch: ",
                      epoch_loss_test,
                      "...")

            break

    '''
    Prediction
    '''
    df_output = pd.DataFrame(columns=['fips', 'Week_end', 'Prediction_1w',
                                      'Prediction_2w', 'Prediction_3w',
                                      'Prediction_4w'])

    test_weeks = df_test[df_test.index.get_level_values('Week_end') \
                       >= pd.to_datetime('2024-01-20')].index.get_level_values('Week_end').unique()
    test_states = df_test.index.get_level_values('fips').unique()

    m_state_dict_main = torch.load(model_file + run.target + '_'  +str(run.lr) + '_' +
                                   str(run.week_range) + '_' + str(run.hidden_layer_size) \
                                   + '_' + str(run.seq_length) + '_' + str(run.num_layers) \
                                   + '_GRU_weights.pt')

    model_main = GRU(run.input_size, run.hidden_layer_size, run.num_layers, run.output_size, run.dropout_rate).to(device)

    model_main.load_state_dict(m_state_dict_main)

    with torch.no_grad():

        for fips in test_states:
            for week in test_weeks:
                seq = df_test[(df_test.index.get_level_values('fips') == fips)\
                 & (df_test.index.get_level_values('Week_end') <= week)][-run.seq_length:].to_numpy()

                seq = torch.tensor(seq).reshape(-1, run.seq_length, run.input_size).to(device)

                model_main.hidden = torch.zeros(run.num_layers, seq.size(0), run.hidden_layer_size).to(device)
                prediction = model_main(seq.float())

                prediction = scaler_target.inverse_transform(prediction.cpu().detach().numpy().reshape(-1, 1))

                dic = {
                    'fips' : fips,
                    'Week_end' : week,
                    'Prediction_1w' : prediction[0].item(),
                    'Prediction_2w' : prediction[1].item(),
                    'Prediction_3w' : prediction[2].item(),
                    'Prediction_4w' : prediction[3].item()

                }

                df_output = pd.concat([df_output, pd.DataFrame([dic])], ignore_index=True)


        df_output.to_csv(prediction_file + run.target + '_'  +str(run.lr) + '_' +
                            str(run.week_range) + '_' + str(run.hidden_layer_size) \
                            + '_' + str(run.seq_length) + '_' + str(run.num_layers) \
                            + '.csv')

        '''
        Evaluation
        '''



        dic_lstm = {
            'Model': 'GRU',
            'lr': run.lr,
            'batch_size': run.batch_size,
            'seq_length': run.seq_length,
            'week_range': run.week_range,
            'hidden_layer_size': run.hidden_layer_size,
            'num_layers': run.num_layers,
            'lossfunc': run.lossfunc,

            'mse_validation': best_mse_numerical,

            'mse_validation_1w': best_mse_numerical_weekly[0],
            'mse_validation_2w': best_mse_numerical_weekly[1],
            'mse_validation_3w': best_mse_numerical_weekly[2],
            'mse_validation_4w': best_mse_numerical_weekly[3],

            'Target': run.target
        }

        print('mse_validation:')
        print(dic_lstm['mse_validation'])


        df_summary = pd.concat([df_summary, pd.DataFrame([dic_lstm])], ignore_index=True)

        df_summary.to_csv(summary_file + 'summarytest0.csv')

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.8 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
  0%|          | 0/50 [00:00<?, ?it/s]

Train Loss:  197.20484140515327
Test Loss:  48.55406130850315


  4%|▍         | 2/50 [00:03<01:22,  1.71s/it]

Train Loss:  135.7801804728806
Test Loss:  39.00799242267385


  6%|▌         | 3/50 [00:04<01:07,  1.45s/it]

...

Counter 4 of 5
Train Loss:  118.8743472439528
Test Loss:  51.67924027889967


 58%|█████▊    | 29/50 [01:36<01:09,  3.33s/it]

Counter 5 of 5
Train Loss:  123.52814664982725
Test Loss:  42.03477039845893
Early stopping with best_loss:  40.477302574552596 and test_loss for this epoch:  42.03477039845893 ...



/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.8 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


mse_validation:
0.9083332743517355


### run best model for 100 times

In [ ]:
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import MinMaxScaler
from torch import nn
from tqdm import trange
from datetime import date, timedelta

from collections import OrderedDict
from collections import namedtuple
from itertools import product

import torch.nn.functional as F


n_result = pd.DataFrame()
data_file = '/content/drive/MyDrive/Flu Forecasting/code/trainingdata_rate0131.csv'
model_file = '/content/drive/MyDrive/Flu Forecasting/GRU/weightselection/'
prediction_file = '/content/drive/MyDrive/Flu Forecasting/GRU/weightselection/'
summary_file ='/content/drive/MyDrive/Flu Forecasting/GRU/weightselection/'
# read data
def perdelta(start, end, delta):
    curr = start
    while curr < end:
        yield curr
        curr += delta
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



params = OrderedDict(
    target = ['rate'], ### Our target variable
    lr = [0.0001],
    batch_size = [2],
    seq_length = [12], # length of input sequence to predict values
    output_size = [4], # we are predicting next four weeks admissions
    num_pred_features = [1],
    week_range = [64], # number of historical data (per state) used to train the model
    input_size = [15], # total number of input features
    hidden_layer_size = [256], #test
    num_layers = [1], #test
    ratio = [0.7],  # ratio of training set and validation set
    num_epochs = [50],
    dropout_rate = [0.8],
    lossfunc = [nn.SmoothL1Loss(beta=0.05, reduction = 'sum'),

                ]
)


class RunBuilder():
    @staticmethod
    def get_runs(params):

        Run = namedtuple('Run', params.keys())

        runs = []
        for v in product(*params.values()):
            runs.append(Run(*v))

        return runs

pd.options.mode.chained_assignment = None

runs = RunBuilder.get_runs(params)

for run in RunBuilder.get_runs(params):
    if run.target == 'rate':
        results_file = 'rate'

    validation_predictions = []
    validation_labels = []
    # read data
    df = pd.read_csv(data_file)
    #df = df.rename(columns={'End Date': 'Week_end', 'State': 'fips'})
    df['Week_end'] = pd.to_datetime(df['Week_end'])
    # too many missing values in State'11'
    df =  df[df['fips'] != 11]
    #df =  df[df['fips'] == 1]
    #df = df[['fips', 'Week_end', 'total_admissions']]
    #df = df.drop(['PRCP_mean', 'SNOW_mean', 'TMAXDELTA', 'TMINDELTA'], axis=1)
    # choose data in 2023-11 as test data  test 10-01
    '''
    start_date1 = pd.to_datetime('2023-11-01') - timedelta(weeks=run.seq_length * 7)
    df_test = df[(df['Week_end'] > start_date1) & (df['Week_end'] < pd.to_datetime('2023-11-26'))] # Use data after 2023-11-01 as testing set
    df_test = df_test.set_index(['fips', 'Week_end'])
    df_test1 = df_test.copy()
    #choose data before 2023-11 as training data and validation data
    df_train = df[df['Week_end'] <= pd.to_datetime('2023-11-01')] # Use data before 11/1 as training set
    df_train = df_train.set_index(['fips', 'Week_end'])
'''

    start_date1 = pd.to_datetime('2024-01-20') - timedelta(weeks=run.seq_length)
    df_test = df[(df['Week_end'] > start_date1) & (df['Week_end'] <pd.to_datetime('2024-01-26'))] # Use data after 2023-11-01 as testing set
    df_test = df_test.set_index(['fips', 'Week_end'])
    df_test1 = df_test.copy()
    #choose data before 2024-01-06 as training data and validation data and after 2022-08-01
    df_train = df[(df['Week_end'] <= pd.to_datetime('2024-01-20'))& (df['Week_end'] >pd.to_datetime('2022-08-01')) ] # Use data before 11/1 as training set

    df_train = df_train.set_index(['fips', 'Week_end'])
    #rescale the data from 0 to 1, normalization
    first_col = df_train.pop(run.target)
    df_train.insert(0, run.target, first_col)
    first_col_2 = df_test.pop(run.target)
    df_test.insert(0, run.target, first_col_2)

    scaler = MinMaxScaler(feature_range=(0, 1))

    scaler.fit(df_train.iloc[:, 1:])
    train_features_normalized = scaler.transform(df_train.iloc[:, 1:])
    test_feature_normalized = scaler.transform(df_test.iloc[:, 1:])

    scaler_target = MinMaxScaler(feature_range=(0, 1))
    scaler_target.fit(np.asarray(df_train.iloc[:, 0]).reshape(-1, 1))
    train_target_normalized = scaler_target.transform(np.asarray(df_train.iloc[:, 0]).reshape(-1, 1))
    test_target_normalized = scaler_target.transform(np.asarray(df_test.iloc[:, 0]).reshape(-1, 1))

    df_train.iloc[:, 1:] = train_features_normalized
    df_train.iloc[:, 0] = train_target_normalized
    df_test.iloc[:, 1:] = test_feature_normalized
    df_test.iloc[:, 0] = test_target_normalized


    for i in range(1000):

      df_output = pd.DataFrame(columns=['fips', 'Week_end', 'Prediction_1w',
                                        'Prediction_2w', 'Prediction_3w',
                                        'Prediction_4w'])

      test_weeks = df_test[df_test.index.get_level_values('Week_end') \
                        >= pd.to_datetime('2024-01-20')].index.get_level_values('Week_end').unique()
      test_states = df_test.index.get_level_values('fips').unique()

      m_state_dict_main = torch.load(model_file + run.target + '_'  +str(run.lr) + '_' +
                                    str(run.week_range) + '_' + str(run.hidden_layer_size) \
                                    + '_' + str(run.seq_length) + '_' + str(run.num_layers) \
                                    + '_GRU_weights.pt')

      model_main = GRU(run.input_size, run.hidden_layer_size, run.num_layers, run.output_size, run.dropout_rate).to(device)

      model_main.load_state_dict(m_state_dict_main)

      with torch.no_grad():

          for fips in test_states:
              for week in test_weeks:
                  seq = df_test[(df_test.index.get_level_values('fips') == fips)\
                  & (df_test.index.get_level_values('Week_end') <= week)][-run.seq_length:].to_numpy()

                  seq = torch.tensor(seq).reshape(-1, run.seq_length, run.input_size).to(device)

                  model_main.hidden = torch.zeros(run.num_layers, seq.size(0), run.hidden_layer_size).to(device)
                  prediction = model_main(seq.float())

                  prediction = scaler_target.inverse_transform(prediction.cpu().detach().numpy().reshape(-1, 1))

                  dic = {
                      'fips' : fips,
                      'Week_end' : week,
                      'Prediction_1w' : prediction[0].item(),
                      'Prediction_2w' : prediction[1].item(),
                      'Prediction_3w' : prediction[2].item(),
                      'Prediction_4w' : prediction[3].item()

                  }

                  df_output = pd.concat([df_output, pd.DataFrame([dic])], ignore_index=True)


          df_output.to_csv(prediction_file + run.target + '_'  +str(run.lr) + '_' +
                              str(run.week_range) + '_' + str(run.hidden_layer_size) \
                              + '_' + str(run.seq_length) + '_' + str(run.num_layers) \
                              + '.csv')
          n_result= pd.concat([n_result, df_output], ignore_index=True)


    n_result.to_pickle(summary_file + 'allresulttest1.pkl')

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.8 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.8 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.8 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout op

In [ ]:
from google.colab import runtime
runtime.unassign()

In [ ]:
n_result

,fips,Week_end,Prediction_1w,Prediction_2w,Prediction_3w,Prediction_4w
0,1,2023-12-16,1.617280,1.146696,0.627724,1.514531
1,2,2023-12-16,0.245919,0.087149,-0.078879,-0.251350
2,4,2023-12-16,3.430712,5.010484,4.956791,4.955416
3,5,2023-12-16,0.488530,0.196350,0.206587,0.519681
4,6,2023-12-16,1.144734,0.942700,0.767006,0.711150
...,...,...,...,...,...,...
49995,51,2023-12-16,1.061623,1.649244,1.543778,2.170187
49996,53,2023-12-16,0.098470,0.038732,-0.212735,-0.420063
49997,54,2023-12-16,0.871705,0.616140,1.013480,1.827708
49998,55,2023-12-16,1.142060,1.293435,1.473662,1.732367
